# ETL PROJECT (RE)

In [7]:
# import field
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import os
import sqlite3

## Logger

In [8]:
# 로그 파일 => etl_project_log.txt
def log_message(message):
    if not 'etl_project_log.txt' in os.listdir():
        with open('etl_project_log.txt', 'w') as f:
            f.write("ETL Project Log\n")
            f.write("=" * 20 + "\n")
    with open('etl_project_log.txt', 'a') as f:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"{current_time} - {message}\n")

## Region DB 설정, CSV로도 export

In [9]:
# 이미 만들어놓은게 있으니까 그걸 가져다쓰자
# 사실 따로 설정할 것도 없네 ... CSV로만 export
conn = sqlite3.connect('World_Economies.db')

query = """
SELECT * FROM Region
"""
region_df = pd.read_sql_query(query, conn)

conn.close()

region_df

,Country,Geograpical Subregion,Intermediary Region,Continental Region
0,Algeria,Northern Africa,None,Africa
1,Egypt,Northern Africa,None,Africa
2,Libya,Northern Africa,None,Africa
3,Morocco,Northern Africa,None,Africa
4,Sudan,Northern Africa,None,Africa
...,...,...,...,...
276,Zanzibar,None,None,Africa
277,East Timor,None,None,Asia
278,Sint Maarten,None,None,Americas
279,São Tomé and Príncipe,None,None,Africa


In [10]:
# CSV로 저장 
region_df.to_csv('Region.csv', index=False)

## Extract

In [24]:
# 추출"만" -> 사람이 인식하는 테이블 형태로 반환
# 전처리됨 !
def extract():
    # 페이지 요청
    response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
    # html 파싱
    soup = BeautifulSoup(response.content, 'html.parser')
    # 테이블 찾기
    gdp_table = soup.find('table', {'class': 'wikitable'})
    # 테이블 행으로 파싱
    rows = gdp_table.find_all('tr')
    
    ### gdp_df 만들기
    
    countries = []
    gdps = []
    years = []

    for row in rows[3:]:
        eles = row.find_all('td')  # td태그 달린 애들 중 [0][1]만 쓸 것임 !
        # 나라 추가
        country = eles[0].text.strip()
        countries.append(country)
        # gdp, 연도 추가
        # - 가 있음 ! => 적절한 처리(아래)
        if eles[1].text.strip().replace(',', '') != '—':
            gdp_forecast = int(eles[1].text.strip().replace(',', ''))
            gdps.append(gdp_forecast)
            year = int(eles[2].text.strip()[-4:])
            years.append(year)
        else:  # 측정이 안됐을 경우
            gdps.append(np.nan)
            years.append(np.nan)

    # 데이터프레임 생성
    gdp_df = pd.DataFrame(
        {
            'Country': countries,
            'GDP': gdps,
            'Year': years
        }
    )

    return gdp_df

gdp_df = extract()

In [25]:
gdp_df

,Country,GDP,Year
0,United States,28781083.0,2024.0
1,China,18532633.0,2024.0
2,Germany,4591100.0,2024.0
3,Japan,4110452.0,2024.0
4,India,3937011.0,2024.0
...,...,...,...
208,Marshall Islands,305.0,2024.0
209,Cook Islands,NaN,NaN
210,Nauru,161.0,2024.0
211,Montserrat,NaN,NaN


## Transform

In [34]:
# GDP 단위 수정 + Region 정보 merge된 df 반환
def transform(gdp_df):
    # billion 단위로 수정
    gdp_df['GDP'] = (gdp_df['GDP']/1000).round(2)
    region_df = pd.read_csv('Region.csv')
    # region 정보 left outer join
    gdp_region_df = pd.merge(gdp_df, region_df, on = 'Country', how = 'left')
    
    return gdp_region_df


gdp_region_df = transform(gdp_df)

## Load

In [35]:
### 저장할 파일의 이름 결정 함수
# 혹시 이름이 있으면 _1, _2 ... 이런식으로
# 저장할 이름을 반환
# filename은 이름.확장자
# 디렉터리는 현재 디렉터리 기준
def get_unique_filename(filename): 
    # 없으면 그냥 이름 그대로 쓰자
    if not os.path.exists(filename):
        return filename
    else: # 있으면
        base, ext = os.path.splitext(filename)
        counter = 1
        new_filename = f"{base}_{counter}{ext}"
        while os.path.exists(new_filename): # 있으면 카운터 계속 올리기
            counter += 1
            new_filename = f"{base}_{counter}{ext}"
        return new_filename

In [36]:
# Load
def load(gdp_region_df):
    # 파일 이름
    filename = 'gdp_region.json'
    filename = get_unique_filename(filename)
    gdp_region_df.to_json(filename, orient = 'records', lines=True)

load(gdp_region_df)

In [ ]:
if __name__ == "__main__":

    ### 결과 출력
    # GDP 100B 넘는 나라들
    gdp_df = get_gdp_from_wiki()
    print('----- Countries whose GDP is over 100B -----')
    print(gdp_df[gdp_df['GDP'] >= 100])
    # Continental Region 별 Top 5
    region_df = get_region_from_wiki()
    gdp_region_df = pd.merge(gdp_df, region_df, on = 'Country', how = 'left')

    print('-----Top 5 Countries by continental region')
    top5s = gdp_region_df.groupby('Continental Region').apply(lambda x: x.nlargest(5, 'GDP')).reset_index(drop=True)
    print(top5s[['Country','GDP','Continental Region']])

    log_message('The result was printed on console')

    ### gdp_region_df을 JSON으로 저장
    filename = 'gdp_region.json'
    filename = get_unique_filename(filename)
    gdp_region_df.to_json(filename, orient = 'records', lines=True)
    log_message(f'{filename} saved')

In [12]:
########## ETL PROJECT GDP


# import field
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import os

### Logger
# 로그 파일 => etl_project_log.txt
def log_message(message):
    if not 'etl_project_log.txt' in os.listdir():
        with open('etl_project_log.txt', 'w') as f:
            f.write("ETL Project Log\n")
            f.write("=" * 20 + "\n")
    with open('etl_project_log.txt', 'a') as f:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"{current_time} - {message}\n")

### E ! 
# 추출"만" -> 사람이 인식하는 테이블 형태로 반환
# 전처리됨 !
def extract():
    # 페이지 요청
    response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
    # html 파싱
    soup = BeautifulSoup(response.content, 'html.parser')
    # 테이블 찾기
    gdp_table = soup.find('table', {'class': 'wikitable'})
    # 테이블 행으로 파싱
    rows = gdp_table.find_all('tr')
    
    ### gdp_df 만들기
    
    countries = []
    gdps = []
    years = []

    for row in rows[3:]:
        eles = row.find_all('td')  # td태그 달린 애들 중 [0][1]만 쓸 것임 !
        # 나라 추가
        country = eles[0].text.strip()
        countries.append(country)
        # gdp, 연도 추가
        # - 가 있음 ! => 적절한 처리(아래)
        if eles[1].text.strip().replace(',', '') != '—':
            gdp_forecast = int(eles[1].text.strip().replace(',', ''))
            gdps.append(gdp_forecast)
            year = int(eles[2].text.strip()[-4:])
            years.append(year)
        else:  # 측정이 안됐을 경우
            gdps.append(np.nan)
            years.append(np.nan)

    # 데이터프레임 생성
    gdp_df = pd.DataFrame(
        {
            'Country': countries,
            'GDP': gdps,
            'Year': years
        }
    )

    return gdp_df


### Transform
# GDP 단위 수정 + Region 정보 merge된 df 반환
def transform(gdp_df):
    # billion 단위로 수정
    gdp_df['GDP'] = (gdp_df['GDP']/1000).round(2)
    region_df = pd.read_csv('Region.csv')
    # region 정보 left outer join
    gdp_region_df = pd.merge(gdp_df, region_df, on = 'Country', how = 'left')
    
    return gdp_region_df


### 저장할 파일의 이름 결정 함수
# 혹시 이름이 있으면 _1, _2 ... 이런식으로
# 저장할 이름을 반환
# filename은 이름.확장자
# 디렉터리는 현재 디렉터리 기준
def get_unique_filename(filename): 
    # 없으면 그냥 이름 그대로 쓰자
    if not os.path.exists(filename):
        return filename
    else: # 있으면
        base, ext = os.path.splitext(filename)
        counter = 1
        new_filename = f"{base}_{counter}{ext}"
        while os.path.exists(new_filename): # 있으면 카운터 계속 올리기
            counter += 1
            new_filename = f"{base}_{counter}{ext}"
        return new_filename


### Load 
# json으로 저장
def load(gdp_region_df):
    # 파일 이름
    filename = 'gdp_region.json'
    filename = get_unique_filename(filename)
    gdp_region_df.to_json(filename, orient = 'records', lines=True)


if __name__ == "__main__":

    ### E
    log_message("E start !")
    gdp_df = extract()
    log_message("E finished !")

    ### T
    log_message("T start !")
    gdp_region_df = transform(gdp_df)
    log_message("T finished !")

    ### L
    log_message("L start !")
    load(gdp_region_df)
    log_message("L finished !")

    # Load 했으니까 다시 불러와야 하나 ... ?
    # 근데 같은 코드스페이스에 이미 올라와있으니까 ... 이거 쓰자 ... ㅋㅋ!

    ### A(?)
    # GDP 100B 넘는 나라들
    print('----- Countries whose GDP is over 100B -----')
    print(gdp_df[gdp_df['GDP'] >= 100])

    # Top 5
    print('----- Top 5 Countries by continental region -----')
    top5s = gdp_region_df.groupby('Continental Region').apply(lambda x: x.nlargest(5, 'GDP')).reset_index(drop=True)
    print(top5s[['Country','GDP','Continental Region']])

----- Countries whose GDP is over 100B -----
          Country       GDP    Year
0   United States  28781.08  2024.0
1           China  18532.63  2024.0
2         Germany   4591.10  2024.0
3           Japan   4110.45  2024.0
4           India   3937.01  2024.0
..            ...       ...     ...
66      Guatemala    110.04  2024.0
67           Oman    108.93  2024.0
68       Bulgaria    107.93  2024.0
69          Kenya    104.00  2024.0
70      Venezuela    102.33  2024.0

[70 rows x 3 columns]
----- Top 5 Countries by continental region -----
             Country       GDP Continental Region
0       South Africa    373.23             Africa
1              Egypt    347.59             Africa
2            Algeria    266.78             Africa
3            Nigeria    252.74             Africa
4           Ethiopia    205.13             Africa
5      United States  28781.08           Americas
6             Brazil   2331.39           Americas
7             Canada   2242.18           Americas


/var/folders/8x/ndvw4k5j1zxgm1gs6gvqf8cm0000gn/T/ipykernel_6329/1997954209.py:137: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top5s = gdp_region_df.groupby('Continental Region').apply(lambda x: x.nlargest(5, 'GDP')).reset_index(drop=True)


# with SQL

In [13]:


# import field
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import os
import sqlite3

### Logger
# 로그 파일 => etl_project_log.txt
def log_message(message):
    if not 'etl_project_log.txt' in os.listdir():
        with open('etl_project_log.txt', 'w') as f:
            f.write("ETL Project Log\n")
            f.write("=" * 20 + "\n")
    with open('etl_project_log.txt', 'a') as f:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"{current_time} - {message}\n")

### E ! 
# 추출"만" -> 사람이 인식하는 테이블 형태로 반환
# 전처리됨 !
def extract():
    # 페이지 요청
    response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
    # html 파싱
    soup = BeautifulSoup(response.content, 'html.parser')
    # 테이블 찾기
    gdp_table = soup.find('table', {'class': 'wikitable'})
    # 테이블 행으로 파싱
    rows = gdp_table.find_all('tr')
    
    ### gdp_df 만들기
    
    countries = []
    gdps = []
    years = []

    for row in rows[3:]:
        eles = row.find_all('td')  # td태그 달린 애들 중 [0][1]만 쓸 것임 !
        # 나라 추가
        country = eles[0].text.strip()
        countries.append(country)
        # gdp, 연도 추가
        # - 가 있음 ! => 적절한 처리(아래)
        if eles[1].text.strip().replace(',', '') != '—':
            gdp_forecast = int(eles[1].text.strip().replace(',', ''))
            gdps.append(gdp_forecast)
            year = int(eles[2].text.strip()[-4:])
            years.append(year)
        else:  # 측정이 안됐을 경우
            gdps.append(np.nan)
            years.append(np.nan)

    # 데이터프레임 생성
    gdp_df = pd.DataFrame(
        {
            'Country': countries,
            'GDP': gdps,
            'Year': years
        }
    )

    return gdp_df


### Transform
# GDP 단위 수정 + Region 정보 merge된 df 반환
def transform(gdp_df):
    # billion 단위로 수정
    gdp_df['GDP'] = (gdp_df['GDP']/1000).round(2)
    # 컬럼 이름 바꾸기
    gdp_df = gdp_df.rename(columns={'GDP': 'GDP_USD_BILLION'})
    # sql버전에서는 그냥 단위만 수정해서 보내주자 
    # A 과정에서 쿼리로 한번에 할 것! 
    return gdp_df


### 저장할 파일의 이름 결정 함수
# 혹시 이름이 있으면 _1, _2 ... 이런식으로
# 저장할 이름을 반환
# filename은 이름.확장자
# 디렉터리는 현재 디렉터리 기준
def get_unique_filename(filename): 
    # 없으면 그냥 이름 그대로 쓰자
    if not os.path.exists(filename):
        return filename
    else: # 있으면
        base, ext = os.path.splitext(filename)
        counter = 1
        new_filename = f"{base}_{counter}{ext}"
        while os.path.exists(new_filename): # 있으면 카운터 계속 올리기
            counter += 1
            new_filename = f"{base}_{counter}{ext}"
        return new_filename


### Load 
# DB에 테이블로 저장
def load(gdp_df):
    conn = sqlite3.connect('World_Economies.db')
    gdp_df.to_sql('Countries_by_GDP',conn, if_exists='replace', index=False)
    conn.close()

if __name__ == "__main__":

    conn = sqlite3.connect('World_Economies.db')

    ### E
    log_message("E start !")
    gdp_df = extract()
    log_message("E finished !")

    ### T
    log_message("T start !")
    gdp_df = transform(gdp_df)
    log_message("T finished !")

    ### L
    log_message("L start !")
    load(gdp_df)
    log_message("L finished !")

    # Load 했으니까 다시 불러와야 하나 ... ?
    # 근데 같은 코드스페이스에 이미 올라와있으니까 ... 이거 쓰자 ... ㅋㅋ!
    # 

    ### A(?)
    # GDP 100B 넘는 나라들
    print('----- Countries whose GDP is over 100B -----')
    query = """
    SELECT * 
    FROM Countries_by_GDP
    WHERE GDP_USD_BILLION >= 100;
    """
    countries_gdp_over_100 = pd.read_sql_query(query, conn)
    print(countries_gdp_over_100)

    ### 지역 별 TOP 5 출력
    print('-----Top 5 Countries by continental region-----')
    query = """
    SELECT COUNTRY, GDP_USD_BILLION, REGION, RANK 
    FROM (
        SELECT COUNTRY, GDP_USD_BILLION, REGION, ROW_NUMBER() OVER (PARTITION BY REGION ORDER BY GDP_USD_BILLION) AS RANK
        FROM (
            SELECT L.Country AS COUNTRY, L.GDP_USD_BILLION AS GDP_USD_BILLION, R."Continental Region" AS REGION
            FROM Countries_by_GDP L
            LEFT OUTER JOIN Region R
            ON L.Country = R.Country
        )
    ) WHERE RANK <= 5;
    """
    top5s = pd.read_sql_query(query, conn)
    print(top5s)

    conn.close()

----- Countries whose GDP is over 100B -----
          Country  GDP_USD_BILLION    Year
0   United States         28781.08  2024.0
1           China         18532.63  2024.0
2         Germany          4591.10  2024.0
3           Japan          4110.45  2024.0
4           India          3937.01  2024.0
..            ...              ...     ...
65      Guatemala           110.04  2024.0
66           Oman           108.93  2024.0
67       Bulgaria           107.93  2024.0
68          Kenya           104.00  2024.0
69      Venezuela           102.33  2024.0

[70 rows x 3 columns]
-----Top 5 Countries by continental region-----
                  COUNTRY  GDP_USD_BILLION    REGION  RANK
0                 Eritrea              NaN    Africa     1
1                Zanzibar              NaN    Africa     2
2   São Tomé and Príncipe             0.75    Africa     3
3                 Comoros             1.42    Africa     4
4           Guinea-Bissau             2.15    Africa     5
5             